<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 가위바위보 분류기를 만들자

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [4]:
!mkdir -p /content/data/rock
!mkdir -p /content/data/paper
!mkdir -p /content/data/scissor
!mkdir -p /content/data/test/rock
!mkdir -p /content/data/test/paper
!mkdir -p /content/data/test/scissor
!wget -O /content/data/rock/rock.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/rock.zip
!wget -O /content/data/paper/paper.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/paper.zip
!wget -O /content/data/scissor/scissor.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/scissor.zip
!wget -O /content/data/test/rock/rock.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_rock.zip
!wget -O /content/data/test/paper/paper.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_paper.zip
!wget -O /content/data/test/scissor/scissor.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_scissor.zip
!unzip /content/data/paper/paper.zip -d /content/data/paper
!unzip /content/data/rock/rock.zip -d /content/data/rock
!unzip /content/data/scissor/scissor.zip -d /content/data/scissor
!unzip /content/data/test/paper/paper.zip -d /content/data/test/paper
!unzip /content/data/test/rock/rock.zip -d /content/data/test/rock
!unzip /content/data/test/scissor/scissor.zip -d /content/data/test/scissor

Archive:  /content/data/rock/rock.zip
  inflating: /content/data/rock/1.jpg  
  inflating: /content/data/rock/10.jpg  
  inflating: /content/data/rock/100.jpg  
  inflating: /content/data/rock/11.jpg  
  inflating: /content/data/rock/12.jpg  
  inflating: /content/data/rock/13.jpg  
  inflating: /content/data/rock/14.jpg  
  inflating: /content/data/rock/15.jpg  
  inflating: /content/data/rock/16.jpg  
  inflating: /content/data/rock/17.jpg  
  inflating: /content/data/rock/18.jpg  
  inflating: /content/data/rock/19.jpg  
  inflating: /content/data/rock/2.jpg  
  inflating: /content/data/rock/20.jpg  
  inflating: /content/data/rock/21.jpg  
  inflating: /content/data/rock/22.jpg  
  inflating: /content/data/rock/23.jpg  
  inflating: /content/data/rock/24.jpg  
  inflating: /content/data/rock/25.jpg  
  inflating: /content/data/rock/26.jpg  
  inflating: /content/data/rock/27.jpg  
  inflating: /content/data/rock/28.jpg  
  inflating: /content/data/rock/29.jpg  
  inflating: /conten

In [6]:
import os
from PIL import Image
import glob
import tensorflow as tf
import numpy as np

1) 데이터 준비 

In [7]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [8]:
image_dir_path = "/content/data/scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")

100  images to be resized.
100  images resized.
가위 이미지 resize 완료!


In [9]:
image_dir_path = "/content/data/rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")

100  images to be resized.
100  images resized.
바위 이미지 resize 완료!


In [10]:
image_dir_path = "/content/data/paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

100  images to be resized.
100  images resized.
보 이미지 resize 완료!


In [ ]:
def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [ ]:
image_dir_path = "/content/data"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

2) 딥러닝 네트워크 설계 

3) 학습

4) 테스트(평가)

5) 개선하기